In [15]:
# !pip install --force-reinstall --no-cache-dir \
#     numpy==2.0.0 \
#     pandas==2.2.2 \
#     scipy==1.11.4 \
#     scikit-learn==1.5.0 \
#     tsfresh==0.21.0 \
#     umap-learn==0.5.5 \
#     opencv-python-headless==4.12.0.88 \
#     --quiet


# New section

In [16]:
#!pip uninstall -y huggingface_hub --quiet
#!pip install --upgrade huggingface_hub --quiet

In [17]:
import os
from huggingface_hub import InferenceClient
from google.colab import userdata


os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

client = InferenceClient(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    token=userdata.get('HUGGINGFACEHUB_API_TOKEN')

)

In [18]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

def generate_metrics():
    df = pd.DataFrame({
        "cpu": np.random.normal(50, 10, 100),
        "memory": np.random.normal(60, 15, 100),
        "disk_io": np.random.normal(100, 25, 100),
    })
    df.iloc[95:100] += np.random.uniform(100, 200)
    return df

def detect_anomalies(df):
    model = IsolationForest(contamination=0.05, random_state=42)
    model.fit(df)
    df["anomaly"] = model.predict(df)
    return df[df["anomaly"] == -1]


In [19]:
def build_messages(anomalies_df):
    return [
        {"role": "system", "content": "You are an expert AIOps assistant."},
        {"role": "user", "content": f"""These are the system anomalies detected using Isolation Forest:

{anomalies_df.to_string(index=False)}

Please explain:
1. Likely root cause
2. Possible impact
3. Recommended fix
"""}
    ]


In [20]:
import time

def evaluate_response(response_text, start_time):
    duration = round(time.time() - start_time, 2)
    words = len(response_text.split())

    keywords = ["cpu", "memory", "disk", "fix", "restart", "resource", "usage"]
    relevance_hits = sum(kw in response_text.lower() for kw in keywords)
    relevance_score = round(relevance_hits / len(keywords), 2)

    return {
        "latency_seconds": duration,
        "word_count": words,
        "relevance_score": relevance_score,
    }


In [21]:
df = generate_metrics()
anomalies = detect_anomalies(df)

if anomalies.empty:
    print("✅ No anomalies detected.")
else:
    print("🚨 Anomalies found. Sending to Mixtral...\n")
    messages = build_messages(anomalies)

    start = time.time()
    response = client.chat_completion(messages=messages)
    reply = response.choices[0].message.content

    print("🧠 Mixtral Response:\n", reply)

    metrics = evaluate_response(reply, start)
    print("\n📊 Evaluation Metrics:")
    for k, v in metrics.items():
        print(f" - {k}: {v}")


🚨 Anomalies found. Sending to Mixtral...

🧠 Mixtral Response:
 Based on the system anomalies detected using Isolation Forest, I'll provide my analysis.

**1. Likely root cause:**

The anomalies detected across multiple metrics (cpu, memory, disk_io) suggest a resource-intensive process or a high load on the system. The metrics are not extremely high, but they are consistently above the normal range, indicating a potential bottleneck.

A likely root cause could be:

- **High CPU usage due to a resource-hungry process:** A process or a set of processes might be consuming excessive CPU resources, causing the system to slow down and impacting other applications.
- **Memory-intensive application:** An application or a service might be consuming a significant amount of memory, leading to high memory usage and potentially causing the system to swap to disk, thus affecting performance.
- **Disk I/O issues:** High disk I/O could be caused by a disk failure, a faulty disk controller, or a disk b